In [5]:
!pip3 install pickle5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 11.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pickle
import pickle5

import gcsfs
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [2]:
# N = 100
base_dir = 'gs://gbi_ml/classification_hackathon/'
base_dir_gcsfs = base_dir[5:]

In [3]:
base_dir_gcsfs

'gbi_ml/classification_hackathon/'

In [4]:
fs = gcsfs.GCSFileSystem()

with fs.open(base_dir+'id2label.pickle', 'rb') as handle:
    id2label = pickle.load(handle)

In [5]:
with fs.open(base_dir+'X_train.pickle', 'rb') as handle:
    train_embs = pickle5.load(handle)

with fs.open(base_dir+'X_test.pickle', 'rb') as handle:
    test_embs = pickle5.load(handle)

In [6]:
test_embs.shape, train_embs.shape

((19481, 512), (308284, 512))

In [7]:
y_train_loaded = pd.read_csv(base_dir+'train_labels.csv')['label']
y_test_loaded = pd.read_csv(base_dir+'test_labels.csv')['label']

In [9]:
index_del = y_train_loaded[y_train_loaded==473].index

In [21]:
index_del = y_test_loaded[y_test_loaded.isin(val_counts[val_counts==1].index)].index

In [22]:
y_test_loaded.drop(index=index_del, inplace=True)

In [23]:
test_embs = np.delete(test_embs, index_del, 0)

In [11]:
y_train_loaded.drop(index=index_del, inplace=True)

In [12]:
train_embs = np.delete(train_embs, index_del, 0)

In [13]:
train_embs.shape, y_train_loaded.shape

((308283, 512), (308283,))

In [18]:
val_counts = y_test_loaded.value_counts()

In [20]:
val_counts[val_counts==1].index

Int64Index([508, 350, 310, 345, 641, 142, 342, 339, 502, 334, 314,  16, 539,
            648, 547, 650, 256, 288,  14, 653, 134,   8, 265, 422, 143,  25,
            210, 596,  65, 203, 440, 576,  60,  58,  53,  92, 457, 387, 172,
            246, 621, 482, 379, 111, 375, 425,  29,  28, 487, 299],
           dtype='int64')

In [24]:
# X_train, X_val, y_train, y_val = train_test_split(train_embs, y_train_loaded,
#                                                   test_size=0.1, random_state = 42, stratify=y_train_loaded)
X_train, X_val, y_train, y_val = train_test_split(test_embs, y_test_loaded,
                                                  test_size=0.1, random_state = 42, stratify=y_test_loaded)

In [ ]:
np.setdiff1d(y_val, y_train)

In [25]:
# Get the train and test data for the training sequence
train_dataset = lgb.Dataset(X_train, label=y_train)
test_dataset = lgb.Dataset(X_val, label=y_val)

parameters = {
    # 'application': 'binary',
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting': 'gbdt',
    # 'num_leaves': 31,
    # 'feature_fraction': 0.5,
    # 'bagging_fraction': 0.5,
    # 'bagging_freq': 20,
    # 'learning_rate': 0.05,
    # 'verbose': 0,
    # 'max_depth': 10,
    # 'device': 'gpu',
    'num_classes': len(id2label),
    'min_data_in_leaf':300,
    'force_col_wise':True
}

# Train the classifier
# classifier = lgb.LGBMClassifier()
# classifier.fit(X_train, y_train, eval_set=(X_val, y_val))
# model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.1, n_estimators=300, device = "gpu")
classifier = lgb.train(
    parameters, train_dataset, valid_sets=[train_dataset, test_dataset],
    #  num_boost_round=5000,
    early_stopping_rounds=10)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 130560
[LightGBM] [Info] Number of data points in the train set: 17487, number of used features: 512
[LightGBM] [Info] Start training from score -2.475515
[LightGBM] [Info] Start training from score -8.670601
[LightGBM] [Info] Start training from score -5.761880
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -6.878841
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -7.130156
[LightGBM] [Info] Start training from score -6.773481
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -7.284306
[LightGBM] [Info] Start training from score -3.937331
[LightGBM] [Info] Start training from score -5.438480
[LightGBM] [Info] Start training from score -4.227949
[LightGBM] [Info] Start training from score -9.076066
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -7.204264
[L

KeyboardInterrupt: 

In [ ]:
y_test_pred = classifier.predict(X_test)

In [ ]:
y_test_pred

In [ ]:
y_test_pred = classifier.predict(train_embs)
report = classification_report(y_test, y_test_pred, output_dict=True, labels=list(id2label.values))
df_report = pandas.DataFrame(report).transpose()
df_report.to_csv(base_dir+'classification_report_test.csv')

model.metadata["framework"] = "lightgbm"
file_name = model.path + ".pkl"
with open(file_name, 'wb') as file:  
    pickle.dump(classifier, file)

In [12]:
from sklearn.metrics import classification_report
x = [[1],[1],[1],[2],[3]]
y_true = [0, 1, 2, 2, 2]
y_pred = [0, 0, 2, 2, 1]
target_names = ['class 0', 'class 1', 'class 2']

In [2]:
report = classification_report(y_true, y_pred, target_names=target_names)

In [4]:
import lightgbm as lgb
classifier = lgb.LGBMClassifier()

In [13]:
classifier.fit(x, y_true)

LGBMClassifier()

In [15]:
y_pred = classifier.predict([[1],[1],[1],[2],[3]])

In [26]:
report = classification_report(y_true, y_pred, output_dict=True, target_names=target_names)

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
report

{'class 0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1},
 'class 1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1},
 'class 2': {'precision': 0.6,
  'recall': 1.0,
  'f1-score': 0.7499999999999999,
  'support': 3},
 'accuracy': 0.6,
 'macro avg': {'precision': 0.19999999999999998,
  'recall': 0.3333333333333333,
  'f1-score': 0.24999999999999997,
  'support': 5},
 'weighted avg': {'precision': 0.36,
  'recall': 0.6,
  'f1-score': 0.4499999999999999,
  'support': 5}}

In [28]:
df_report = pd.DataFrame(report).transpose()

In [29]:
df_report

,precision,recall,f1-score,support
class 0,0.00,0.000000,0.00,1.0
class 1,0.00,0.000000,0.00,1.0
class 2,0.60,1.000000,0.75,3.0
accuracy,0.60,0.600000,0.60,0.6
macro avg,0.20,0.333333,0.25,5.0
weighted avg,0.36,0.600000,0.45,5.0
